In [1]:
import pandas as pd

# 0. Cargar datos

In [2]:
# Este es el dataframe con los numeros de estación mas su geolocalización

estaciones = pd.read_csv('stations.csv')

In [12]:
estaciones.head()

,id,lat,lon,name
0,1,19.433571,-99.167809,1 RIO SENA-RIO BALSAS
1,2,19.431386,-99.171695,2 RIO GUADALQUIVIR-RIO BALSAS
2,3,19.431655,-99.158668,3 REFORMA-INSURGENTES
3,4,19.428309,-99.171713,4 RIO NILO-RIO PANUCO
4,5,19.429720,-99.169363,5 RIO PANUCO-RIO TIBER


## 0.1 Estaciones fuera de mapa

In [3]:
# Hay varias estaciones fuera de los mapas.
# Esto inserta la estacion 1002. Coordenadas hipoteticas basadas en 
# punto central de todos los destinos de esa estacion

estacion_1002 = {'id': 1002, 'lat': 19.42527836428572, 'lon': -99.1695673232143, 
                 'name': '1002 La estación fantásma del Profesor Layton'}
 
estaciones = estaciones.append(estacion_1002, ignore_index=True)

# 1.0 Crear Dataframe con distancias entre estaciones

In [7]:
# https://kanoki.org/2019/02/14/how-to-find-distance-between-two-points-based-on-latitude-and-longitude-using-python-and-sql/

# Esta es una función matematica para encontrar la distancia entre dos puntos de una esfera.
# Usar esto, en vez de una función mas complicada para rutas entre calles, fue para simplificar
# Calculos y visualizaciones.

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
# Esta función hace lo siguiente: Crea un dataframe desde la lista de estaciones.
# De cada par de estaciones retiro/arribo
# calcula la distancia entre ellas (con 0 para retiro y arribo a la misma)
# Y crea columnas con la distancia entre cada par, y las combinaciones de pares.

# El paquete tqdm genera barras de progreso. No es esencial pero ayuda con el aspecto psicológico
# de ver progreso al procesar datasets muy grandes.

from tqdm import tqdm

def fn_dist_estaciones(dataframe):
    id_estaciones = estaciones.id.unique()
    comb_estaciones = []
    par_estaciones = []

    for origen in tqdm(range(len(id_estaciones))):
        origen_id = id_estaciones[origen]
        for destino in range(len(id_estaciones)):
            destino_id = id_estaciones[destino]
            par_estaciones = [origen_id, destino_id]
            comb_estaciones.append(par_estaciones)

    distancia_list = []
    
    for combo in tqdm(range(len(comb_estaciones))):
        lon1 = dataframe.loc[dataframe['id'] == comb_estaciones[combo][0], 'lon']
        lat1 = dataframe.loc[dataframe['id'] == comb_estaciones[combo][0], 'lat']
        lon2 = dataframe.loc[dataframe['id'] == comb_estaciones[combo][1], 'lon']
        lat2 = dataframe.loc[dataframe['id'] == comb_estaciones[combo][1], 'lat']
        distancia = haversine(lon1, lat1, lon2, lat2)
        distancia_list.append(distancia)
            
    distancia_dict = {'retiro_arribo': comb_estaciones, 'distancia': distancia_list}  

    df_distancias = pd.DataFrame(distancia_dict)
    df_distancias['est_retiro'] = df_distancias.apply(lambda x: (x['retiro_arribo'][0]), axis =  1)
    df_distancias['est_arribo'] = df_distancias.apply(lambda x: (x['retiro_arribo'][1]), axis =  1)
#    df_distancias.drop('retiro_arribo', axis=1)
    
    return df_distancias

In [9]:
estaciones_distancia = fn_dist_estaciones(estaciones)

100%|██████████| 231361/231361 [06:36<00:00, 583.12it/s]


In [11]:
estaciones_distancia.head()

,retiro_arribo,distancia,est_retiro,est_arribo
0,"[1, 1]",0.000000,1,1
1,"[1, 2]",0.474436,1,2
2,"[1, 3]",0.981942,1,3
3,"[1, 4]",0.714133,1,4
4,"[1, 5]",0.458207,1,5


# 1.1 Exportar

In [ ]:
estaciones_distancia.to_csv('df_distancia_estaciones.csv', index=False)